# Hello, welcome to PCWO.

In this module, we will walk through running (and creating) a fully functional Photonic Crystal Waveguide Optimizer (PCWO) script. 

The PCWO code base utilizes the Photonic Crystal Waveguide (PCW) simulation tool provided by MIT Photonic Bands (MPB). PCWO and our ongoing research into PCW optimization is supported by Sebastian Schulz, Jeremy Upham, and Robert Boyd through the University of Ottawa Quantum Photonics group. This code is released under the GNU General Public License-3.0.

MPB: http://ab-initio.mit.edu/wiki/index.php/MIT_Photonic_Bands

Citation: S Billings, SA Schulz, J Upham, RW Boyd, "Application-Tailored Optimization of Photonic Crystal Waveguides", J Opt. N, (2016)

GNU GPL-3.0: https://opensource.org/licenses/GPL-3.0




This notebook will demonstrate:

1) How to integrate our W1 waveguide mpb .ctl script to be used by the PCWO packages.

2) Generate reusable PCW Simulation objects

3) Initialize both pareto, and objective functions to be used as optimization conditions

4) Configure and run the Strength Pareto Evolutionary Algorithm (SPEA)

5) Configure and run the Stochastic Gradient Descent Algorithm (SGD)



First off, we have to import a few programs that are available in the PCWO backend folder.


In [1]:
from backend import constraints 
from backend.experiment import W1Experiment
from backend.spea_optimizer import SpeaOptimizer
from backend.sgd_optimizer import StochasticGradientDescentOptimizer
from backend.photonicCrystalDesign import PhCWDesign
from backend.paretoFunctions import ParetoMaxFunction
from backend.objectiveFunctions import WeightedSumObjectiveFunction

Next, we record the paths to an mpb installation, a Waveguide File, and an output for our mpb computations which will be parsed.
	Note, this output file will be reused every time our algorithms evaluate a PhC design.

In [2]:
# absolute path to the mpb executable
mpb = "/Users/sean/documents/mpb-1.5/mpb/mpb"
#mpb = "/your/path/to/mpb"

# mpb = "mpb" 
# depending on how mpb is installed, the assignment mpb = "mpb" may be all that is required
# this was the case on an Ubuntu dist that I was testing on

# absolute path to the mpb waveguide input ctl
inputFile = "/Users/sean/UniversityOfOttawa/Photonics/PCWO/W1_2D_v04.ctl.txt"
#inputFile = "/your/path/to/W1_2D_v04.ctl.txt"

# absolute path to the mpb output file
outputFile = "/Users/sean/UniversityOfOttawa/Photonics/PCWO/optimizerTestFile.txt"
#outputFile = "/your/path/to/optimizerTestFile.txt"

We automate our mpb executions (usually done with terminal) by wrapping them in an object called an Experiment. 

The various PCWO algorithms use these experiment objects to simulate a given instance of a Photonic Crystal Design.

In [3]:
# we define a general experiment object
# that we reuse whenever we need to make a command-line mpb call
# see experiment.py for functionality
experiment = W1Experiment(mpb, inputFile, outputFile)
experiment.setCalculationType('4') # accepts an int from 0 to 5
experiment.setBand(23)

Now, we initialize our parameter map, load a set of predefined constraints from constraints.py in the backend folder, and store these values in a PhCWDesign object, with a score initialized to 0. Note, 'score' is not immediately useful for Optimization routines that use Pareto Functions. However, score is utilized by the Objective Functions used in Differential Evolution and Gradient Descent algorithms.

In [4]:
paramMap = {}
paramMap["s1"] = 0 # First row vertical shift
paramMap["s2"] = 0 # Second row vertical shift
paramMap["s3"] = 0 # Third row vertical shift
#paramMap["p1"] = 0 # First row horizontal shift
#paramMap["p2"] = 0 # Second row horizontal shift
#paramMap["p3"] = 0 # Third row horizontal shift
paramMap["r0"] = 0.3 # Default air-hole radius
paramMap["r1"] = 0.3 # Default first row radius
paramMap["r2"] = 0.3 # Default second row radius
paramMap["r3"] = 0.3 # Default third row radius

# see constraints.py
constraintFunctions = [constraints.latticeConstraintsLD]

pcw = PhCWDesign(paramMap, 0, constraintFunctions)

SPEA requires a pareto function in order to compare pcw designs. Because we are going to use a pareto function in order to compare PCW, we must establish whether we want to minimize or maximize objectives. We do this by using a key map. Similarily to in paramMap, only the terms that are added to this dictionary will be optimized for. 

We use a ParetoMaxFunction by convention. This function will invert objectives in the key map that have been specified as "min". In other words, if key_map["loss"] = "min", then 1/loss will be evaluated and maximized.


In [5]:
#Initialize pareto function

key_map = {}
key_map["ng0"] = "max"
key_map["loss_at_ng0"] = "min"

pareto_function = ParetoMaxFunction(experiment, key_map)

Next, we define the parameters for SPEA. Because each PhCW takes a long time to simulate in mpb, one should be conservative with the population size and number of generations.

In [6]:
#SPEA parameters

max_generation = 10 # number of iterations of the SPEA algorithm
population_size = 10 # number of solutions to consider 
pareto_archive_size = 8 # number of solutions to store after each generation
tournament_selection_rate  = 5 # number of solutions to consider in crossover/mutation

We initialize our population randomly, and then run our SPEA optimizer.

In [ ]:
# Run the optimizer
population = SpeaOptimizer.createPopulation(population_size, pcw)

optimizer = SpeaOptimizer(pareto_function)

print "Starting SPEA"

optimizer.optimize(population,max_generation,tournament_selection_rate, pareto_archive_size)

print "\nSPEA solutions generated"

Starting SPEA


Generation 0



Now, we have an optimized set of solutions that is stored in population. The theme of our system is to combine (hybridize) optimization methods, so we will apply Stochastic Gradient Descent SGD to our population to further improve the quality of our solutions. 

To apply a gradient descent method, we first have to specify and configure the objective function that we will use.



In [ ]:
# specify the weights for the WeightedSumObjectiveFunction

w1 = 0 # bandwidth weight
w2 = 0 # group index weight
w3 = 0 # average loss weight
w4 = 1 #0.6 # BGP weight
w5 = 0.0 # loss at ngo (group index) weight
w6 = 0 # delay weight

weights = [ w1, w2, w3, w4, w5, w6]


#Initialize objective function
objective_function = WeightedSumObjectiveFunction(weights, experiment)

Now we initialize a second optimizer and pass our configured objective function to it. Afterwards we initialize run time parameters for the SGD algorithm, and then launch the optimization

In [ ]:
max_iterations = 10 # number of iterations of the DE alg
fault_tolerance = 3 # limit for un-fruitful 'direction' attempts

optimizer2 = StochasticGradientDescentOptimizer(objFunc)

results = optimizer2.optimize(population, max_iterations, fault_tolerance)

print "\nStochastic Gradient Descent solutions generated"

Thank you for using PCWO